## NLP Assignment-3 Part-2
### Name : Ishaan Sinha
### Roll number : 21CS30064

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/My Drive')

In [3]:
# Install necessary libraries
!pip install accelerate transformers pandas torch

In [4]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, BitsAndBytesConfig
import torch

In [5]:
train = pd.read_csv('/content/drive/My Drive/NLP_ass_train.tsv', sep='\t')
val = pd.read_csv('/content/drive/My Drive/NLP_ass_valid.tsv', sep='\t')
test = pd.read_csv('/content/drive/My Drive/NLP_ass_test.tsv', sep='\t')

train.columns = ['text', 'label']
val.columns = ['text', 'label']
test.columns = ['text', 'label']

#### Function to find out number of common sentences

In [6]:
def Intersection_of_datasets(s1, s2):
  return len(set(s1).intersection(set(s2)))

In [7]:
train_dataset = list(train['text'])
val_dataset = list(val['text'])
test_dataset = list(test['text'])

print("Number of common sentences (train, test): ", Intersection_of_datasets(train_dataset, test_dataset))
# print("Number of common sentences (val, test): ", Intersection_of_datasets(val_dataset, test_dataset))

Number of common sentences (train, test):  5


In [ ]:
# Installing dependencies
!pip install -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


#### PEFT-QLoRa Model

In [ ]:
Bits_And_Bytes_Configuration = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(f"google/flan-t5-base", quantization_config=Bits_And_Bytes_Configuration, device_map={"":0})
tokenizer = T5Tokenizer.from_pretrained(f"google/flan-t5-base")
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear4bit(in_features=768, out_features=768, bias=False)
              (k): Linear4bit(in_features=768, out_features=768, bias=False)
              (v): Linear4bit(in_features=768, out_features=768, bias=False)
              (o): Linear4bit(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear4bit(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear4bit(in_features=768, out_features=2048, bias=Fa

In [13]:
import re

# Identify attention-related layers in the model for LoRA fine-tuning
def locate_attention_layers(model):
    attention_pattern = re.compile(r'((self\.)?attention\.)|((self\.)?attn\.)', re.IGNORECASE)
    attention_layers = [name for name, module in model.named_modules() if re.search(attention_pattern, name)]
    return attention_layers

attention_layers = locate_attention_layers(model)
target_modules = attention_layers

In [ ]:
from peft import LoraConfig, get_peft_model

# Define the LoRA configuration for targeted parameter-efficient fine-tuning
Lora_configuration = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, Lora_configuration)

In [15]:
# Define label mappings for classifying statements
label_mapping = {
    'normal': 0,
    'hatespeech': 1,
    'offensive': 2
}

In [16]:
import torch.nn.functional as F

# Convert predicted labels to a one-hot encoded format with logits
def create_one_hot_labels(preds, num_classes):
    pred_ids = [label_mapping[label] for label in preds]
    one_hot_encoded = F.one_hot(torch.tensor(pred_ids), num_classes=num_classes)
    log_logits = torch.log10(one_hot_encoded.float() + 1e-6)  # Add small epsilon to avoid log(0)
    return log_logits

In [17]:
from torch.utils.data import Dataset, DataLoader

class TextClassificationDataset(Dataset):
    def __init__(self, tokenizer, data, max_token_length, class_count, answer_token_length, intro, response_options, text_template):
        self.sentences = list(data['text'])
        self.target_labels = list(data['label'])
        self.tokenizer = tokenizer
        self.class_count = class_count
        self.max_token_length = max_token_length
        self.answer_token_length = answer_token_length
        self.intro = intro
        self.response_options = response_options
        self.text_template = text_template

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        target_label = self.target_labels[index]

        # Create the input prompt using the template
        formatted_input = self.text_template.substitute(
            intro=self.intro, prompt=sentence, options=self.response_options
        )

        # Tokenize input prompt and label text
        label_encoding = self.tokenizer(
            target_label,
            add_special_tokens=True,
            max_length=self.answer_token_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        prompt_encoding = self.tokenizer(
            formatted_input,
            add_special_tokens=True,
            max_length=self.max_token_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        label_index = torch.tensor([label_mapping[target_label]])

        input_ids = prompt_encoding['input_ids']
        attention_mask = prompt_encoding['attention_mask']

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label_ids': torch.tensor(label_encoding['input_ids'], dtype=torch.long),
            'labels': label_index
        }

In [18]:
# New function for label mapping based on specific keywords
def clean_and_map_labels(predictions):
    mapped_labels = []
    for prediction in predictions:
        if 'hate' in prediction:
            mapped_labels.append('hatespeech')
        elif 'norm' in prediction:
            mapped_labels.append('normal')
        elif 'off' in prediction:
            mapped_labels.append('offensive')
        else:
            mapped_labels.append('normal')  # Default to normal if uncertain

    return mapped_labels

In [19]:
from string import Template

# Configuration for tokenization and data handling
max_token_length = 512
answer_token_length = 2
class_count = 3
batch_size = 32
intro = 'Determine whether the sentence is normal, hatespeech, or offensive.'
options = ''
template = Template('$intro\nSentence: $prompt$options\nAnswer:')

# Instantiate the datasets and dataloaders
train_ds = TextClassificationDataset(tokenizer, train, max_token_length, class_count, answer_token_length, intro, options, template)
val_ds = TextClassificationDataset(tokenizer, val, max_token_length, class_count, answer_token_length, intro, options, template)
test_ds = TextClassificationDataset(tokenizer, test, max_token_length, class_count, answer_token_length, intro, options, template)

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

#### Fine-tuned Model

In [20]:
import torch.nn as nn

class FineTunedFlanT5Model(nn.Module):
  def __init__(self, model, class_count):
    super(FineTunedFlanT5Model, self).__init__()
    self.model = model
    self.class_count = class_count

  def forward(self, input_ids, attention_mask, labels):
    outputs = self.model(
                  input_ids=input_ids,
                  attention_mask=attention_mask,
                  labels=labels
                )

    logits = outputs['logits']
    preds__ = torch.argmax(logits, dim=2)
    bsize = preds__.shape[0]
    preds_ = [tokenizer.decode(preds__[i], skip_special_tokens=True) for i in range(bsize)]
    preds = clean_and_map_labels(preds_)
    preds_logits = create_one_hot_labels(preds, class_count)
    return torch.tensor(preds_logits, dtype=torch.float)

In [21]:
# Wrap model with fine-tuned configuration
peft_model = FineTunedFlanT5Model(model, class_count)

In [22]:
lr = 2e-5
epochs = 10

In [23]:
import torch.optim as optim

# Optimizer and loss function
optimizer = optim.Adam(peft_model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

#### Training Model

In [24]:
from tqdm import tqdm

prev_val_acc = -1
patience = 3
k = 0

for epoch in range(epochs):
    train_loss = 0.0
    print(f"Epoch {epoch+1}/{epochs}")
    peft_model.train()

    for batch in tqdm(train_dataloader, desc="Training batches"):
      input_ids = batch["input_ids"].squeeze(1).to(device)
      attention_mask = batch["attention_mask"].squeeze(1).to(device)
      label_ids = batch["label_ids"].squeeze(1).to(device)
      labels = batch["labels"].squeeze(1).to(device)

      optimizer.zero_grad()
      with torch.cuda.amp.autocast():
        output_logits = peft_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=label_ids
                  ).to(device)

        loss = loss_fn(output_logits, labels)

      # print(loss)
      loss.requires_grad = True
      loss.backward()
      optimizer.step()
      # scaler.update()

      train_loss += loss.item()

    train_loss /= len(train_dataloader)
    model.eval()

    val_loss = 0.0
    val_samples = 0
    val_correct = 0

    with torch.no_grad():
      for batch_idx, batch in tqdm(enumerate(val_dataloader)):
          input_ids = batch["input_ids"].squeeze(1).to(device)
          attention_mask = batch["attention_mask"].squeeze(1).to(device)
          label_ids = batch["label_ids"].squeeze(1).to(device)
          labels = batch["labels"].squeeze(1).to(device)

          with torch.cuda.amp.autocast():
            output_logits = peft_model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=label_ids
                      ).to(device)

            loss = loss_fn(output_logits, labels)

          # labels = labels.type_as(outputs)

          val_loss += loss.item()

          predictions = torch.argmax(output_logits, axis=1)

          num_correct = sum(1 for a, b in zip(predictions, labels) if a == b)
          num_samples = predictions.shape[0]

          val_samples += num_samples
          val_correct += num_correct

    val_acc = val_correct / val_samples
    val_loss /= len(val_dataloader)

    if val_acc < prev_val_acc:
      k += 1

    if k == patience:
      break

    prev_val_acc = val_acc

    print(f"Train loss : {train_loss}, Val loss : {val_loss}, Val acc : {val_acc}")

Epoch 1/10


Training batches:   0%|          | 0/481 [00:00<?, ?it/s]<ipython-input-17-9725ccd6e9b9>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label_ids': torch.tensor(label_encoding['input_ids'], dtype=torch.long),
<ipython-input-24-f1d848511336>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants

Train loss : 3.5664876349998362, Val loss : 3.604329734552102, Val acc : 0.40655908381051536
Epoch 2/10


Training batches: 100%|██████████| 481/481 [08:33<00:00,  1.07s/it]
61it [00:59,  1.03it/s]


Train loss : 3.566842007290053, Val loss : 3.6043296876500865, Val acc : 0.40655908381051536
Epoch 3/10


...

Train loss : 3.5666648142055264, Val loss : 3.509042824404772, Val acc : 0.40655908381051536
Epoch 9/10


Training batches: 100%|██████████| 481/481 [08:33<00:00,  1.07s/it]
61it [00:59,  1.03it/s]


Train loss : 3.5671963800759423, Val loss : 3.509042797045263, Val acc : 0.40655908381051536
Epoch 10/10


Training batches: 100%|██████████| 481/481 [08:33<00:00,  1.07s/it]
61it [00:59,  1.03it/s]

Train loss : 3.567373561759996, Val loss : 3.604329734552102, Val acc : 0.40655908381051536


### Evaluation of Whole Test Dataset

In [25]:
from sklearn.metrics import f1_score
import torch
from tqdm import tqdm

def evaluate_model(dataloader, model, device):
    """
    Evaluates the model on the given dataloader and returns accuracy and macro F1 score.

    Args:
        dataloader (DataLoader): DataLoader containing the dataset to evaluate.
        model (nn.Module): The model to evaluate.
        device (torch.device): Device to run the evaluation on (e.g., 'cuda' or 'cpu').

    Returns:
        float: Test accuracy.
        float: Macro F1 score.
    """
    model.eval()  # Set model to evaluation mode
    sample_count = 0
    correct_count = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader), desc='Performing Inference'):
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            label_ids = batch['label_ids'].squeeze(1).to(device)
            labels = batch['labels'].squeeze(1).to(device)

            # Get model predictions
            logits = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)
            predictions = torch.argmax(logits, dim=1)

            # Track predictions and labels for F1 score calculation
            predicted_labels.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

            # Calculate batch accuracy
            correct_predictions = sum(1 for a, b in zip(predictions, labels) if a == b)
            batch_size = predictions.shape[0]

            sample_count += batch_size
            correct_count += correct_predictions

    # Compute evaluation metrics
    macro_f1_score = f1_score(true_labels, predicted_labels, average='macro')
    accuracy = correct_count / sample_count

    return accuracy, macro_f1_score

# Use the function to evaluate on the test dataset
test_accuracy, macro_f1_score = evaluate_model(test_dataloader, peft_model, device)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Macro F1 Score: {macro_f1_score:.4f}")

Performing Inference: 0it [00:00, ?it/s]<ipython-input-17-9725ccd6e9b9>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label_ids': torch.tensor(label_encoding['input_ids'], dtype=torch.long),
<ipython-input-20-3d35483ce482>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(preds_logits, dtype=torch.float)
Performing Inference: 61it [01:37,  1.60s/it]

Test Accuracy: 0.4789
Macro F1 Score: 0.3758


### Evaluation of 50 sentences of Test Dataset

In [27]:
def evaluate_model(dataloader, model, device, sample_limit=50):
    """
    Evaluates the model on a subset of the given dataloader (up to sample_limit)
    and returns accuracy and macro F1 score.

    Args:
        dataloader (DataLoader): DataLoader containing the dataset to evaluate.
        model (nn.Module): The model to evaluate.
        device (torch.device): Device to run the evaluation on (e.g., 'cuda' or 'cpu').
        sample_limit (int): Maximum number of samples to evaluate.

    Returns:
        float: Test accuracy.
        float: Macro F1 score.
    """
    model.eval()  # Set model to evaluation mode
    sample_count = 0
    correct_count = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader), desc='Performing Inference'):
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            label_ids = batch['label_ids'].squeeze(1).to(device)
            labels = batch['labels'].squeeze(1).to(device)

            # Get model predictions
            logits = model(input_ids=input_ids, attention_mask=attention_mask, labels=label_ids)
            predictions = torch.argmax(logits, dim=1)

            # Track predictions and labels for F1 score calculation
            predicted_labels.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

            # Calculate batch accuracy
            correct_predictions = sum(1 for a, b in zip(predictions, labels) if a == b)
            batch_size = predictions.shape[0]

            sample_count += batch_size
            correct_count += correct_predictions

            # Stop evaluation after reaching the sample limit
            if sample_count >= sample_limit:
                break

    # Only consider the first 50 samples for metric calculation
    true_labels = true_labels[:sample_limit]
    predicted_labels = predicted_labels[:sample_limit]
    accuracy = correct_count / sample_limit
    macro_f1_score = f1_score(true_labels, predicted_labels, average='macro')

    return accuracy, macro_f1_score

# Use the function to evaluate on a limited number of samples in the test dataset
test_accuracy, macro_f1_score = evaluate_model(test_dataloader, peft_model, device, sample_limit=50)
print(f"Test Accuracy (50 samples): {test_accuracy:.4f}")
print(f"Macro F1 Score (50 samples): {macro_f1_score:.4f}")

Performing Inference: 0it [00:00, ?it/s]<ipython-input-17-9725ccd6e9b9>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label_ids': torch.tensor(label_encoding['input_ids'], dtype=torch.long),
<ipython-input-20-3d35483ce482>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(preds_logits, dtype=torch.float)
Performing Inference: 1it [00:03,  3.20s/it]

Test Accuracy (50 samples): 0.7600
Macro F1 Score (50 samples): 0.4416
